In [1]:
# YouTube RAG QA Bot - Complete Implementation with LangSmith
# ===========================================================

import os
import glob
import warnings
import asyncio
import numpy as np
import librosa
from dotenv import load_dotenv
import whisper
import gradio as gr
import nest_asyncio
from typing import List, Dict, Any, Optional
import logging
import uuid
from datetime import datetime
from yt_dlp import YoutubeDL 

nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)

# LangChain Core
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from pydantic import BaseModel, PrivateAttr

# Mistral Integration
from mistralai import Mistral, UserMessage

# LangSmith Integration
from langsmith import Client
from langchain.callbacks import LangChainTracer
from langchain.schema import HumanMessage, AIMessage

In [2]:
# ==================== Configuration ====================
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT", "thisonecanbefinal3")

if not MISTRAL_API_KEY:
    raise ValueError("❌ MISTRAL_API_KEY not found in environment variables")

# LangSmith Configuration
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
    print("✅ LangSmith tracing enabled")
else:
    print("⚠️ LANGCHAIN_API_KEY not found - LangSmith tracing disabled")

# Directory Configuration
BASE_DIR = "youtube_rag_project"
AUDIO_DIR = os.path.join(BASE_DIR, "audio")
TRANSCRIPTS_DIR = os.path.join(BASE_DIR, "transcripts")
CHROMA_DIR = os.path.join(BASE_DIR, "chroma_db")
TTS_DIR = os.path.join(BASE_DIR, "tts")

# Model Configuration
WHISPER_MODEL_SIZE = "base"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
VIDEO_LIMIT = 30
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

# Create directories
for dir_path in [AUDIO_DIR, TRANSCRIPTS_DIR, TTS_DIR, CHROMA_DIR]:
    os.makedirs(dir_path, exist_ok=True)


✅ LangSmith tracing enabled


In [3]:
# ==================== LangSmith Integration ====================
class LangSmithTracker:
    """LangSmith integration for monitoring and evaluation"""
    
def __init__(self):
    self.client = None
    if LANGCHAIN_API_KEY:
        try:
            self.client = Client(api_key=LANGCHAIN_API_KEY)
            from langsmith.schemas import TracerSessionCreate
            
            session_name = f"youtube-rag-session-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
            session = self.client.create_session(
                TracerSessionCreate(name=session_name)
            )
            self.session_id = session.id
            
            logging.info(f"✅ LangSmith client initialized, session: {session_name}")
        except Exception as e:
            logging.error(f"LangSmith initialization error: {e}")
            self.client = None
    
def log_interaction(self, question: str, answer: str, sources: List[str] = None):
    """Log user interactions for evaluation"""
    if not self.client:
        return

    try:
        self.client.create_run(
            name="youtube_rag_query",
            run_type="chain",
            inputs={"question": question},
            outputs={"answer": answer, "sources": sources or []},
            project_name=LANGCHAIN_PROJECT,
            session_id=self.session_id,
            start_time=datetime.now(),
            end_time=datetime.now()
        )
        logging.info("✅ Interaction logged to LangSmith")
    except Exception as e:
        logging.error(f"LangSmith logging error: {e}")
        
    
    def create_evaluation_dataset(self, qa_pairs: List[Dict]):
        """Create evaluation dataset in LangSmith"""
        if not self.client:
            return None
            
        try:
            dataset_name = f"youtube-rag-eval-{datetime.now().strftime('%Y%m%d')}"
            dataset = self.client.create_dataset(
                dataset_name=dataset_name,
                description="YouTube RAG Bot evaluation dataset"
            )
            
            for qa_pair in qa_pairs:
                self.client.create_example(
                    dataset_id=dataset.id,
                    inputs={"question": qa_pair["question"]},
                    outputs={"expected_answer": qa_pair["answer"]}
                )
            
            logging.info(f"✅ Created evaluation dataset: {dataset_name}")
            return dataset
            
        except Exception as e:
            logging.error(f"Dataset creation error: {e}")
            return None


In [4]:
# ==================== Mistral LLM Wrapper ====================
class MistralLLM(LLM, BaseModel):
    """Custom Mistral LLM wrapper for LangChain"""
    api_key: str
    model: str = "mistral-large-latest"
    temperature: float = 0.0
    max_tokens: int = 1000
    _mistral_client: Any = PrivateAttr()

    def __init__(self, **data: Any):
        super().__init__(**data)
        self._mistral_client = Mistral(api_key=self.api_key)

    @property
    def _llm_type(self) -> str:
        return "mistral"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        try:
            messages = [UserMessage(content=prompt)]
            response = self._mistral_client.chat.complete(
                model=self.model,
                messages=messages,
                temperature=self.temperature,
                max_tokens=self.max_tokens
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            logging.error(f"Mistral API error: {e}")
            return f"❌ Error: Unable to process request"

In [5]:
# ==================== Data Processing Pipeline ====================
class YouTubeDataProcessor:
    """Handles YouTube video downloading and transcription"""
    
    def __init__(self):
        self.whisper_model = None
    
    def download_videos(self, channel_url: str, limit: int = VIDEO_LIMIT) -> bool:
        """Download YouTube videos as audio files"""
        existing_files = glob.glob(os.path.join(AUDIO_DIR, "*.mp3"))
        
        if len(existing_files) >= limit:
            logging.info(f"✅ Found {len(existing_files)} audio files, skipping download")
            return True

        logging.info(f"📥 Downloading {limit} videos from channel...")
        
        ydl_opts = {
            "format": "bestaudio/best",
            "postprocessors": [{
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "128",
            }],
            "playlistend": limit,
            "outtmpl": os.path.join(AUDIO_DIR, "%(title)s.%(ext)s"),
            "ignoreerrors": True,
            "quiet": True,
            "no_warnings": True
        }
        
        try:
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([channel_url])
            logging.info("✅ Download completed")
            return True
        except Exception as e:
            logging.error(f"Download error: {e}")
            return False
    
    def transcribe_audio_files(self) -> Dict[str, str]:
        """Transcribe all audio files to text"""
        if self.whisper_model is None:
            logging.info("🧠 Loading Whisper model...")
            self.whisper_model = whisper.load_model(WHISPER_MODEL_SIZE)
        
        transcripts = {}
        audio_files = glob.glob(os.path.join(AUDIO_DIR, "*.mp3"))
        
        for audio_file in audio_files:
            title = os.path.splitext(os.path.basename(audio_file))[0]
            transcript_path = os.path.join(TRANSCRIPTS_DIR, f"{title}.txt")
            
            # Check if transcript already exists
            if os.path.exists(transcript_path):
                with open(transcript_path, "r", encoding="utf-8") as f:
                    transcripts[title] = f.read()
                continue
            
            logging.info(f"🎙️ Transcribing: {title[:50]}...")
            try:
                result = self.whisper_model.transcribe(audio_file)
                transcript_text = result["text"].strip()
                transcripts[title] = transcript_text
                
                # Save transcript
                with open(transcript_path, "w", encoding="utf-8") as f:
                    f.write(transcript_text)
                    
            except Exception as e:
                logging.error(f"Transcription error for {title}: {e}")
        
        logging.info(f"✅ Processed {len(transcripts)} transcripts")
        return transcripts


In [6]:
# ==================== RAG Vector Store ====================
class RAGVectorStore:
    """Handles vector database creation and management"""
    
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(
            model_name=EMBEDDING_MODEL,
            model_kwargs={'device': 'cpu'}
        )
        self.vectorstore = None
    
    def create_or_load_vectorstore(self, transcripts: Dict[str, str]) -> Chroma:
        """Create or load existing vector database"""
        
        # Check if vectorstore exists
        if os.path.exists(os.path.join(CHROMA_DIR, "chroma.sqlite3")):
            logging.info("✅ Loading existing vectorstore...")
            self.vectorstore = Chroma(
                persist_directory=CHROMA_DIR, 
                embedding_function=self.embeddings
            )
            return self.vectorstore
        
        logging.info("📚 Creating new vectorstore...")
        
        # Text splitting and chunking
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            separators=["\n\n", "\n", ". ", " "]
        )
        
        documents = []
        for title, transcript in transcripts.items():
            if not transcript.strip():
                continue
                
            chunks = text_splitter.split_text(transcript)
            for i, chunk in enumerate(chunks):
                doc = Document(
                    page_content=chunk,
                    metadata={
                        "source": title,
                        "chunk_id": i,
                        "total_chunks": len(chunks)
                    }
                )
                documents.append(doc)
        
        if not documents:
            raise ValueError("No documents to vectorize")
        
        # Create vectorstore
        self.vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=CHROMA_DIR
        )
        
        self.vectorstore.persist()
        logging.info(f"✅ Vectorstore created with {len(documents)} chunks")
        return self.vectorstore

In [7]:
# ==================== Agent Tools ====================
def create_vector_search_tool(vectorstore: Chroma) -> Tool:
    """Create vector search tool for the agent"""
    
    def vector_search(query: str) -> str:
        """Search through YouTube video transcripts"""
        try:
            retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 3}
            )
            docs = retriever.get_relevant_documents(query)
            
            if not docs:
                return "❌ No relevant information found in the video transcripts."
            
            # Format results
            results = []
            for i, doc in enumerate(docs, 1):
                source = doc.metadata.get("source", "Unknown")
                content = doc.page_content[:400]  # Limit content
                results.append(f"**Source {i}: {source}**\n{content}...")
            
            return "\n\n".join(results)
            
        except Exception as e:
            logging.error(f"Vector search error: {e}")
            return f"❌ Search error: {str(e)}"
    
    return Tool(
        name="vector_search",
        description="Search through YouTube video transcripts for relevant information. Input should be a search query.",
        func=vector_search
    )

def create_transcription_tool() -> Tool:
    """Create audio transcription tool"""
    
    def transcribe_audio(audio_path: str) -> str:
        """Transcribe audio file to text"""
        try:
            if not os.path.exists(audio_path):
                return "❌ Audio file not found"
            
            model = whisper.load_model(WHISPER_MODEL_SIZE)
            result = model.transcribe(audio_path)
            return f"🎙️ Transcription: {result['text']}"
            
        except Exception as e:
            logging.error(f"Transcription error: {e}")
            return f"❌ Transcription failed: {str(e)}"
    
    return Tool(
        name="transcribe_audio",
        description="Transcribe audio input to text. Input should be audio file path.",
        func=transcribe_audio
    )

In [8]:
# ==================== Main RAG Agent ====================
class YouTubeRAGAgent:
    """Main RAG agent with tools, memory, and LangSmith tracking"""
    
    def __init__(self, mistral_api_key: str, vectorstore: Chroma):
        # Initialize LLM
        self.llm = MistralLLM(api_key=mistral_api_key)
        
        # Initialize LangSmith tracker
        self.langsmith_tracker = LangSmithTracker()
        
        # Initialize memory
        self.memory = ConversationBufferWindowMemory(
            memory_key="chat_history",
            return_messages=True,
            k=6  # Keep last 3 exchanges
        )
        
        # Initialize tools
        self.tools = [
            create_vector_search_tool(vectorstore),
            create_transcription_tool()
        ]
        
        # Create callbacks for LangSmith
        callbacks = []
        if LANGCHAIN_API_KEY:
            callbacks.append(LangChainTracer(project_name=LANGCHAIN_PROJECT))
        
        # Create agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True, 
            handle_parsing_errors=True,
            max_iterations=6, #increased iterations so to get better reply
            callbacks=callbacks
        )
    
    def query(self, question: str) -> Dict[str, Any]:
        """Process user question and return answer with metadata"""
        try:
            # Enhance prompt for better responses
            enhanced_prompt = f"""
            You are a friendly and knowledgeable AI assistant that answers questions based **only** on YouTube video transcripts.

            TOOLS:
            You have access to the `vector_search` tool to look up relevant information.

            RULES:
            1.Always use the `vector_search` tool before answering. Never answer from general knowledge.
            2.Base your answer **only on the transcript chunks returned** by the tool. If nothing is returned, respond honestly: "Sorry, I couldn't find any info on that topic in the videos I have."
            3.Be conversational and clear, like you're explaining to a curious friend.
            4.Always include the video title(s) from the source in your response.
            5.Never invent video titles or make up facts. If you're unsure, say so.

            User Question: {question}
            """
            
            response = self.agent.run(enhanced_prompt)
            
            # Extract sources from the response (basic implementation)
            sources = self._extract_sources(response)
            
            return {
                "answer": response,
                "sources": sources,
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            logging.error(f"Query processing error: {e}")
            error_response = f"❌ Sorry, I encountered an error processing your question: {str(e)}"
            return {
                "answer": error_response,
                "sources": [],
                "timestamp": datetime.now().isoformat()
            }
    
    def _extract_sources(self, response: str) -> List[str]:
        """Extract source information from response"""
        sources = []
        lines = response.split('\n')
        for line in lines:
            if "Source" in line and ":" in line:
                source = line.split(":", 1)[1].strip().replace("**", "")
                if source and source not in sources:
                    sources.append(source)
        return sources
    
    def clear_memory(self):
        """Clear conversation memory"""
        self.memory.clear()
        logging.info("🧹 Memory cleared")
    
    def get_conversation_history(self) -> List[Dict]:
        """Get formatted conversation history"""
        history = []
        if hasattr(self.memory, 'chat_memory'):
            for message in self.memory.chat_memory.messages:
                if isinstance(message, HumanMessage):
                    history.append({"role": "user", "content": message.content})
                elif isinstance(message, AIMessage):
                    history.append({"role": "assistant", "content": message.content})
        return history


In [9]:
# ==================== Text-to-Speech ====================
async def text_to_speech_async(text: str) -> Optional[str]:
    """Convert text to speech using Edge TTS"""
    try:
        import edge_tts
        
        # Clean text for TTS
        clean_text = text.replace("**", "").replace("*", "")
        #chatgpt fix
        #if len(clean_text) > 500:
        #    clean_text = clean_text[:500] + "..."
        
        output_path = os.path.join(TTS_DIR, "response.mp3")
        communicate = edge_tts.Communicate(clean_text, voice="en-US-JennyNeural")
        await communicate.save(output_path)
        return output_path
    except Exception as e:
        logging.error(f"TTS Error: {e}")
        return None

def text_to_speech(text: str) -> Optional[str]:
    """Synchronous wrapper for TTS"""
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    return loop.run_until_complete(text_to_speech_async(text))

In [10]:
# ==================== Main Processing Pipeline ====================
def process_question(text_input: str, audio_input) -> tuple:
    """Main processing pipeline for questions"""
    question = ""
    
    # Process audio input first
    if audio_input is not None:
        try:
            # Load and process audio
            audio_data, sr = librosa.load(audio_input, sr=16000)
            
            # Transcribe using Whisper
            model = whisper.load_model(WHISPER_MODEL_SIZE)
            result = model.transcribe(audio_data.astype(np.float32))
            question = result["text"].strip()
            
            logging.info(f"🎙️ Audio transcribed: {question}")
            
        except Exception as e:
            logging.error(f"Audio processing error: {e}")
            return f"❌ Audio processing failed: {str(e)}", None, ""
    
    # Use text input if no audio question
    if not question and text_input:
        question = text_input.strip()
    
    # Validate input
    if not question:
        return "⚠️ Please provide a question via text or audio.", None, ""
    
    # Get answer from RAG agent
    try:
        result = rag_agent.query(question)
        answer = result["answer"]
        sources = result["sources"]
        
        # Format sources for display
        sources_text = ""
        if sources:
            sources_text = "\n\n**Sources:**\n" + "\n".join([f"• {source}" for source in sources])
        
        # Generate audio response
        audio_path = text_to_speech(answer)
        
        return answer + sources_text, audio_path, question
        
    except Exception as e:
        logging.error(f"Processing error: {e}")
        return f"❌ Error processing question: {str(e)}", None, question

def get_conversation_history():
    """Get formatted conversation history"""
    try:
        history = rag_agent.get_conversation_history()
        if not history:
            return "No conversation history yet."
        
        formatted_history = []
        for entry in history[-6:]:  # Show last 6 entries
            role_icon = "🧑" if entry["role"] == "user" else "🤖"
            formatted_history.append(f"{role_icon} **{entry['role'].title()}**: {entry['content']}")
        
        return "\n\n".join(formatted_history)
    except Exception as e:
        return f"Error retrieving history: {e}"

In [11]:
# ==================== Initialize System ====================
def initialize_system():
    """Initialize the complete RAG system"""
    global rag_agent
    
    logging.info("🚀 Initializing YouTube RAG Bot...")
    
    # Initialize data processor
    processor = YouTubeDataProcessor()
    
    # Download videos (Life Noggin channel)
    channel_url = "https://www.youtube.com/@lifenoggin/videos"
    processor.download_videos(channel_url, VIDEO_LIMIT)
    
    # Transcribe videos
    transcripts = processor.transcribe_audio_files()
    
    if not transcripts:
        raise ValueError("No transcripts found. Please check video download and transcription.")
    
    # Create vector store
    vector_store_manager = RAGVectorStore()
    vectorstore = vector_store_manager.create_or_load_vectorstore(transcripts)
    
    # Initialize RAG agent
    rag_agent = YouTubeRAGAgent(MISTRAL_API_KEY, vectorstore)
    
    logging.info("✅ System initialized successfully!")

In [12]:
# ==================== Initialize Gradio ====================
def create_gradio_interface():
    """Create the Gradio web interface with LangSmith monitoring and images"""

    with gr.Blocks(
        title="🧠 Blocko Bot The open source alternative of knowledge", 
        theme=gr.themes.Soft(),
        css="""
        /* Cyan gradient background */
        .gradio-container {
            background: linear-gradient(135deg, #00c6ff 0%, #0072ff 100%) !important;
            min-height: 100vh;
            padding: 40px;
            width: 100vw;
            max-width: 100% ! important;
        }        
        .gradio-container {
        max-width: 1400px !important;
        background-color: rgba(255, 255, 255, 0.85);
        padding: 20px;
        border-radius: 16px;
        }
        .langsmith-status {
            background: linear-gradient(90deg, #10b981, #059669);
            color: white;
            padding: 10px;
            border-radius: 8px;
            margin: 10px 0;
            text-align: center;
        }
        """
    ) as interface:
        
        # ✅ Banner Image (Local)
        gr.Image(value="picture/channels4_banner.jpg", label=None, interactive=False, show_label=False)



        langsmith_status = "🟢 LangSmith Tracking: ON" if LANGCHAIN_API_KEY else "🟡 LangSmith Tracking: OFF"

        gr.Markdown(f"""
        ## 🧠 YouTube RAG QA Bot with LangChain Agents

        <div class="langsmith-status">
        {langsmith_status} | Project: {LANGCHAIN_PROJECT}
        </div>

        This bot allows you to ask questions about YouTube videos using advanced AI techniques:
        - 🤖 **LangChain Agents** with tools & memory
        - 📚 **Vector Search** via Chroma
        - 🎙️ **Voice Input & Output** (Whisper + Edge TTS)
        - 📊 **LangSmith Monitoring**
        """)

        with gr.Tab("💬 Chat Interface"):
            with gr.Row():
                with gr.Column(scale=2):
                    # ✅ Updated Bot Avatar Image Path
                    gr.Image(value="picture/blocko.png", label="🤖 Hi there! I'm Blocko", interactive=False)

                    text_input = gr.Textbox(
                        label="💬 Ask your question",
                        placeholder="e.g., What happens to your body in space?",
                        lines=3
                    )

                    audio_input = gr.Audio(
                        label="🎙️ Or ask with your voice",
                        type="filepath"
                    )

                    with gr.Row():
                        ask_btn = gr.Button("🚀 Ask Question", variant="primary", size="lg")
                        clear_btn = gr.Button("🧹 Clear Memory", variant="secondary")

                with gr.Column(scale=3):
                    response_text = gr.Textbox(
                        label="🤖 AI Response",
                        lines=12,
                        max_lines=25
                    )

                    response_audio = gr.Audio(
                        label="🔊 Listen to Response",
                        type="filepath"
                    )

                    last_question = gr.Textbox(
                        label="❓ Last Question",
                        lines=1,
                        interactive=False
                    )

        with gr.Tab("📊 Conversation History"):
            with gr.Column():
                refresh_history_btn = gr.Button("🔄 Refresh History", variant="secondary")
                conversation_history = gr.Textbox(
                    label="📜 Recent Conversations",
                    lines=15,
                    max_lines=25,
                    interactive=False
                )

        with gr.Tab("⚙️ System Info"):
            gr.Markdown("""
            ## 🧩 System Architecture


            ### Configuration
            - **Model**: Mistral Large Latest
            - **Embedding**: all-MiniLM-L6-v2
            - **Whisper**: base
            - **Vector DB**: ChromaDB
            - **LangSmith Project**: final3
            """)

        # Event handlers
        ask_btn.click(
            fn=process_question,
            inputs=[text_input, audio_input],
            outputs=[response_text, response_audio, last_question]
        )

        clear_btn.click(
            fn=lambda: (rag_agent.clear_memory(), "Memory cleared! 🧹"),
            inputs=[],
            outputs=[conversation_history]
        )

        refresh_history_btn.click(
            fn=get_conversation_history,
            inputs=[],
            outputs=[conversation_history]
        )

        # Example questions
        gr.Examples(
            examples=[
                ["Is AI dangerous?"],
                ["How much caffeine per day is recommended?"],
                ["What is Stress?"]
            ],
            inputs=text_input
        )

    return interface


In [13]:
# ==================== Main Execution ====================
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    
    try:
        # Initialize the system
        initialize_system()
        
        # Create and launch interface
        app = create_gradio_interface()
        app.launch(
            server_name="0.0.0.0",
            server_port=7860,
            share=False,  # Creates public link
            debug=True
        )
        
    except Exception as e:
        logging.error(f"Failed to start application: {e}")
        print(f"❌ Error: {e}")

INFO:root:🚀 Initializing YouTube RAG Bot...
INFO:root:✅ Found 30 audio files, skipping download
INFO:root:🧠 Loading Whisper model...
INFO:root:✅ Processed 30 transcripts

INFO:datasets:PyTorch version 2.7.1 available.
INFO:datasets:TensorFlow version 2.19.0 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:✅ Loading existing vectorstore...
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:root:✅ System initialized successfully!


* Running on local URL:  http://0.0.0.0:7860


INFO:httpx:HTTP Request: GET http://localhost:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://localhost:7860/ "HTTP/1.1 200 OK"


* To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:root:🎙️ Audio transcribed: How many hours per day should I sleep?


...

Could not parse LLM output: `❌ Error: Unable to process request`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:

ERROR:root:Mistral API error: API error occurred: Status 429
{"object":"error","message":"Service tier capacity exceeded for this model.","type":"invalid_request_error","param":null,"code":null}


Could not parse LLM output: `❌ Error: Unable to process request`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Thought: Do I need to use a tool? No

AI: Based on the information from the video transcripts, AI technology is advancing rapidly, and it's possible that AI robots could become a part of our daily lives, even as roommates. For example, Tesla is developing a humanoid robot that could handle various tasks like babysitting, walking the dog, getting groceries, and even being a friend. This information is from the video titled "Are You SURE You're Ready for the AI Roommate？".

However, integrating such advanced technology into our homes raises important questions and challenges. There are ethical considerations, such as whether intelligent machines should be granted certain rights, and practical concerns about how these technologies will affect our daily routines and interactions.

So, while AI has the potential to make our lives easier, it also brings up significant issues that we need to be prepared for.

> Finished chain.
Keyboard interruption in main thread... closing server.
